In [1]:
!pip install requests pandas

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ---------------------------------------- 61.0/61.0 kB 3.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.2/11.5 MB 5.3 MB/s eta 0:00:03
   - -------------------------------------- 0.5/11.5 MB 6.4 MB/s eta 0:00:02
   --- ------------------------------------ 1.1/11.5 MB 8.5 MB/s eta 0:00:02
   ------- -------------------------------- 2.1/11.5 MB 12.4 MB/s eta 0:00:01
   ---------- ----------------------------- 3.0/11.5 MB 13.6 MB/s eta 0:00:01
   -------------- ------------------------- 4.1/11.5 MB 15.3 MB/s eta 0:00:01
   ------------------- -------------------- 5.5/11.5 MB 17.7 MB/s eta 0:00:01
   ------

In [1]:
import requests
import pandas as pd

API_KEY = '77ba26a94bf747d61761544063606374da01af62'
# Adjust the URL to fetch block-level data within a specific county in Pennsylvania
# Here, 'tract:*' is replaced with 'block:*' and we specify a county using '&in=county:003'
URL = 'https://api.census.gov/data/2020/dec/pl?get=P1_001N,NAME&for=block:*&in=state:42+county:101&key=' + API_KEY

response = requests.get(URL)

if response.status_code == 200:
    try:
        data = response.json()
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('Decoding JSON has failed')
        print(response.text)
        exit()

    headers = data.pop(0)
    df = pd.DataFrame(data, columns=headers)
    # Assuming the API response includes 'block', 'tract', 'county', and 'state' columns
    # Adjust column renaming as necessary based on the actual API response
    df.rename(columns={'P1_001N': 'total', 'NAME': 'geoname'}, inplace=True)
    # Ensure 'total' column is of integer type
    df['total'] = df['total'].astype(int)
    # Construct 'geoid' with state, county, tract, and block for unique identification at the block level
    df['geoid'] = df['state'] + df['county'] + df['tract'] + df['block']
    # Ensure 'geoid' and 'geoname' are treated as strings
    df['geoid'] = df['geoid'].astype(str)
    df['geoname'] = df['geoname'].astype(str)
    df = df[['geoid', 'geoname', 'total']]
    csv_file_path = 'population_2020_blocks_new.csv'
    df.to_csv(csv_file_path, index=False)
    print(f"Data successfully saved to {csv_file_path}")
else:
    print(f"Failed to fetch data: HTTP status code {response.status_code}")
    print("Response text:", response.text)


Data successfully saved to population_2020_blocks_new.csv
